In [1]:
(require srfi/1)
(require plot/no-gui)

# Constants

In [2]:
(define prefound-primes '(1009 1013 1019
                          10007 10009 10037
                          100003 100019 100043
                          1000003 1000033 1000037
                          1000000007 1000000009 1000000021
                          10000000019 10000000033 10000000061
                          100000000003 100000000019 100000000057
                          1000000000039 1000000000061 1000000000063))

In [3]:
(define prefound-primes '(1009 1013 1019
                          10007 10009 10037
                          100003 100019 100043
                          1000003 1000033 1000037)) ; abridged for testing
                          

In [4]:
(define timestoavg 100000)

In [5]:
(define fermat-times 25)

In [6]:
(define (square x)
  (* x x))

(define (divides? a b)
  (= (remainder b a) 0))

(define (prime? n)
  (= n (smallest-divisor n)))

(define (find-divisor n test-divisor)
  (cond ((> (square test-divisor) n)
         n)
        ((divides? test-divisor n)
         test-divisor)
        (#t (find-divisor
               n
               (advancemethod test-divisor)))))

# 1-21
> Use the smallest-divisor procedure to find the smallest divisor of each of the following numbers: 199, 1999, 19999.

In [7]:
(define (smallest-divisor n)
  (find-divisor n 2)) ; stock method
  ;(find-divisor-integrated n 2)) ; hopefully faster
(define (prime-method x)
  (prime? x))
  ;(fast-prime? x fermat-times))

In [8]:
(define (advancemethod x)
  ;(next x)) ; 1-23 and forward
  (+ x 1)) ; prior to 1-23

In [9]:
(list 199 (smallest-divisor 199) 1999 (smallest-divisor 1999) 19999 (smallest-divisor 19999))

'(199 199 1999 1999 19999 7)

# 1-22
> write a procedure search-for-primes that checks the primality
of consecutive odd integers in a specified range. Use your procedure to find
the three smallest primes larger than 1000; larger than 10,000; larger than
100,000; larger than 1,000,000.

Times were too inconsistent, so I made an averaged version.

In [10]:
(define (avg-timed-prime-test n)
  (newline)
  (display n)
  (avg-start-prime-test n (current-inexact-milliseconds) 0 timestoavg))

(define (avg-start-prime-test n start-time total-time iter)
  (if (prime-method n)
      (let* ((this-time (- (current-inexact-milliseconds)
                          start-time))
            (new-total-time (+ total-time this-time)))
        (if (> iter 0)
          (avg-start-prime-test n (current-inexact-milliseconds) new-total-time (- iter 1))
          (list n (avg-report-prime (* 1.0 (/ new-total-time timestoavg))))))
      #f))
(define (avg-report-prime elapsed-time)
  (display " *** ")
  (display elapsed-time)
  elapsed-time)

(define (avg-search-for-primes minimum goal)
  (define m (if (even? minimum)
                (+ minimum 1)
                (minimum)))
  (define answer (avg-search-for-primes-iter m '() goal))
  (newline)
  answer)
(define (avg-search-for-primes-iter n lst goal)
  (if (= goal 0)
      lst
      (let ((x (avg-timed-prime-test n)))
        (if (not (equal? x #f))
            (avg-search-for-primes-iter (+ n 2) (cons x lst) (- goal 1))
            (avg-search-for-primes-iter (+ n 2) lst goal)))))

In [11]:
(define prime-times-set (append-map (lambda n (avg-search-for-primes (car n) 3)) '(10 100 1000 10000 100000 1000000)))


11 *** 0.00036046630859375
13 *** 0.00026357421875
15
17 *** 0.000304208984375

101 *** 0.00051772705078125
103 *** 0.00069150634765625
105
107 *** 0.0005234228515625

1001
1003
1005
1007
1009 *** 0.0012924853515625
1011
1013 *** 0.00126191650390625
1015
1017
1019 *** 0.00126515869140625

10001
10003
10005
10007 *** 0.003709169921875
10009 *** 0.00513427978515625
10011
10013
10015
10017
10019
10021
10023
10025
10027
10029
10031
10033
10035
10037 *** 0.00377608154296875

100001
100003 *** 0.0111643408203125
100005
100007
100009
100011
100013
100015
100017
100019 *** 0.011252275390625
100021
100023
100025
100027
100029
100031
100033
100035
100037
100039
100041
100043 *** 0.01136404052734375

1000001
1000003 *** 0.03510486328125
1000005
1000007
1000009
1000011
1000013
1000015
1000017
1000019
1000021
1000023
1000025
1000027
1000029
1000031
1000033 *** 0.03489558837890625
1000035
1000037 *** 0.03479798095703125


In [12]:
prime-times-set

'((17 0.000304208984375) (13 0.00026357421875) (11 0.00036046630859375) (107 0.0005234228515625) (103 0.00069150634765625) (101 0.00051772705078125) (1019 0.00126515869140625) (1013 0.00126191650390625) (1009 0.0012924853515625) (10037 0.00377608154296875) (10009 0.00513427978515625) (10007 0.003709169921875) (100043 0.01136404052734375) (100019 0.011252275390625) (100003 0.0111643408203125) (1000037 0.03479798095703125) (1000033 0.03489558837890625) (1000003 0.03510486328125))

In [13]:
(parameterize ([plot-x-transform  log-transform]
                 [plot-x-ticks      (log-ticks)]
               [plot-y-transform  log-transform]
                 [plot-y-ticks      (log-ticks)])
(plot-pict (list (lines prime-times-set #:style 'short-dash
                        #:label "time to verify prime")
                 (points prime-times-set)
                 (function (lambda (x) (* (sqrt x) 0.00002))
                           #:color 'blue #:style 'dot
                           #:label "sqrt(n)")
                 ;(function (lambda (x) (expt 2 n)))
                 )))

(plot-pict '(prog #<procedure:...e/no-gui/plot2d.rkt:144:8> 400) 400 400 400 0 '() #f #f '#(#(11 1000037) #(4894474295696533/73786976294838206464 5059143974154551/144115188075855872)) #<procedure:...lot2d/plot-area.rkt:202:4> #<procedure:...lot2d/plot-area.rkt:713:4> '#(0 0 1))

It takes roughly $\sqrt{10}$ times longer to verify a prime 10 times higher.

# 1.23

define a procedure next that returns 3 if its input is equal
to 2 and otherwise returns its input plus 2. Modify the smallest-divisor
procedure to use `(next test-divisor)` instead of `(+ test-divisor 1)`.

In [14]:
;; We'll be using prefound primes which is larger than the original problem definition,
;;   so let's update this dataset before changing methods
(define prime-times-set (map avg-timed-prime-test prefound-primes))
prime-times-set


1009 *** 0.0012706298828125
1013 *** 0.00131550048828125
1019 *** 0.00127566650390625
10007 *** 0.0037794580078125
10009 *** 0.00367267822265625
10037 *** 0.0036737548828125
100003 *** 0.011170068359375
100019 *** 0.01115712890625
100043 *** 0.01119401611328125
1000003 *** 0.03471720703125
1000033 *** 0.0349194873046875
1000037 *** 0.03480815185546875

'((1009 0.0012706298828125) (1013 0.00131550048828125) (1019 0.00127566650390625) (10007 0.0037794580078125) (10009 0.00367267822265625) (10037 0.0036737548828125) (100003 0.011170068359375) (100019 0.01115712890625) (100043 0.01119401611328125) (1000003 0.03471720703125) (1000033 0.0349194873046875) (1000037 0.03480815185546875))

In [15]:
(define (next n)
  (if (= n 2)
      3
      (+ n 2)))

In [16]:
(define (advancemethod x)
  (next x)) ; 1-23 and forward
  ;(+ x 1)) ; prior to 1-23

In [17]:
;; Using prefound primes now
(define prime-times-next (map avg-timed-prime-test prefound-primes))
prime-times-next


1009 *** 0.0009265478515625
1013 *** 0.0008765380859375
1019 *** 0.00086964599609375
10007 *** 0.002254970703125
10009 *** 0.002259970703125
10037 *** 0.00225693359375
100003 *** 0.0066411181640625
100019 *** 0.00665362548828125
100043 *** 0.00665562255859375
1000003 *** 0.02043483154296875
1000033 *** 0.0203653515625
1000037 *** 0.0204219970703125

'((1009 0.0009265478515625) (1013 0.0008765380859375) (1019 0.00086964599609375) (10007 0.002254970703125) (10009 0.002259970703125) (10037 0.00225693359375) (100003 0.0066411181640625) (100019 0.00665362548828125) (100043 0.00665562255859375) (1000003 0.02043483154296875) (1000033 0.0203653515625) (1000037 0.0204219970703125))

In [18]:
(parameterize ([plot-x-transform  log-transform]
                 [plot-x-ticks      (log-ticks)]
               [plot-y-transform  log-transform]
                 [plot-y-ticks      (log-ticks)])
(plot-pict (list (lines prime-times-set #:style 'short-dash
                        #:label "time to verify prime")
                 (points prime-times-set)
                 (lines prime-times-next #:style 'short-dash
                        #:label "time with next procedure"
                        #:color 'green)
                 (points prime-times-next)
                 ;(function (lambda (x) (* (sqrt x) 0.000015))
                 ;          #:color 'blue #:style 'dot
                 ;          #:label "sqrt(n)")
                 ;(function (lambda (x) (expt 2 n)))
                 )))

(plot-pict '(prog #<procedure:...e/no-gui/plot2d.rkt:144:8> 400) 400 400 400 0 '() #f #f '#(#(1009 1000037) #(8021068562333811/9223372036854775808 5032428480427501/144115188075855872)) #<procedure:...lot2d/plot-area.rkt:202:4> #<procedure:...lot2d/plot-area.rkt:713:4> '#(0 0 1))

Comparing (1000000 0.0347) for the old algorithm with (1000000 0.02), it also runs at $n * \sqrt{10}$, but takes about 3/5th the time. The (next)
function should allow for skipping about half the iterations necessary, which
is why I'm surprised it doesn't run in 1/2 the time. I wonder if the repeated
conditionals involved slow it down enough to be noticeable. Doesn't seem
likely. I'm going to look at what others say about this.

*Later...*

I was wrong, it *was* the conditional. I'll try inlining the code and see what can be gained.

In [19]:
(define (find-divisor-integrated n test-divisor)
  ; when first run, runs logic for test-divisor = 2, then proceeds to odd looping
  (define (fdi-iter test-divisor)
    (cond ((> (square test-divisor) n)
           n)
          ((divides? test-divisor n)
           test-divisor)
          (#t (fdi-iter
               (+ test-divisor 2)))))
  (if (divides? test-divisor n) ;; Assuming test-divisor is 2
      test-divisor
      (fdi-iter (+ test-divisor 1))))

In [20]:
(define (smallest-divisor n)
  ;(find-divisor n 2)) ; stock method
  (find-divisor-integrated n 2)) ; hopefully faster

In [21]:
(define prime-times-fdi (map avg-timed-prime-test prefound-primes))
prime-times-fdi


1009 *** 0.00065838134765625
1013 *** 0.00065428466796875
1019 *** 0.000653828125
10007 *** 0.00163012451171875
10009 *** 0.00163196044921875
10037 *** 0.001629169921875
100003 *** 0.00473072998046875
100019 *** 0.00473059814453125
100043 *** 0.00472817138671875
1000003 *** 0.01454884033203125
1000033 *** 0.01452201171875
1000037 *** 0.014510576171875

'((1009 0.00065838134765625) (1013 0.00065428466796875) (1019 0.000653828125) (10007 0.00163012451171875) (10009 0.00163196044921875) (10037 0.001629169921875) (100003 0.00473072998046875) (100019 0.00473059814453125) (100043 0.00472817138671875) (1000003 0.01454884033203125) (1000033 0.01452201171875) (1000037 0.014510576171875))

In [22]:
(parameterize ([plot-x-transform  log-transform]
                 [plot-x-ticks      (log-ticks)]
               [plot-y-transform  log-transform]
                 [plot-y-ticks      (log-ticks)])
(plot-pict (list (lines prime-times-set #:style 'short-dash
                        #:label "time to verify prime")
                 (points prime-times-set)
                 (lines prime-times-next #:style 'short-dash
                        #:label "time with next procedure"
                        #:color 'green)
                 (points prime-times-next)
                 (lines prime-times-fdi #:style 'short-dash
                        #:label "time with inlined function"
                        #:color 'blue)
                 (points prime-times-fdi)
                 ;(function (lambda (x) (* (sqrt x) 0.000015))
                 ;          #:color 'blue #:style 'dot
                 ;          #:label "sqrt(n)")
                 ;(function (lambda (x) (expt 2 n)))
                 )))

(plot-pict '(prog #<procedure:...e/no-gui/plot2d.rkt:144:8> 400) 400 400 400 0 '() #f #f '#(#(1009 1000037) #(6030500045034189/9223372036854775808 5032428480427501/144115188075855872)) #<procedure:...lot2d/plot-area.rkt:202:4> #<procedure:...lot2d/plot-area.rkt:713:4> '#(0 0 1))

NOTE: In the .scm version, this speedup was more drastic, running at twice the speed of the previous generation. This one is less drastic and it's inconsistent whether the speedup shows at all per rendering.

# 1.24
Modify the `timed-prime-test procedure` of Exercise 1.22 to use
`fast-prime?` (the Fermat method), and test each of the 12 primes you found in
that exercise. Since the Fermat test has $Θ(\log{n})$ growth, how would you expect
the time to test primes near 1,000,000 to compare with the time needed to
test primes near 1000? Do your data bear this out? Can you explain any
discrepancy you find?